In [3]:
!pip install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.2.2

In [4]:
import os
# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")

In [5]:
##第一步 读取mfd.md文件 

In [5]:
import re
import hashlib    

In [6]:
def parse_civil_code_md(file_path):
    with open(file_path, 'r',encoding="utf-8") as f:
        content = f.read()
    articles = []
    # 当前所在的标题层级
    current_part = ""      # 编（## 中华人民共和国民法典）
    current_chapter = ""   # 章（### （二）物权编）
    current_section = ""   # 节（#### 第一章 一般规定）
    current_subsection = "" # 子节（##### 第一节 一般规定）
    current_subsubsection = "" # 子子节（###### 一、动产质权）

    print("开始解析文件...")
    print("-" * 40)
    #按行分割
    lines = content.split("\n")
    i=0
    while i < len(lines):
        line = lines[i].strip()
        # 跳过空行
        if not line:
            i+=1
            continue
        # 检测标题层级
        if line.startswith('## '):
            # 二级标题：编
            current_part = line[3:].strip()
            # 遇到新编时，重置所有下级标题
            current_chapter = ""
            current_section = ""
            current_subsection = ""
            current_subsubsection = ""
            print(f"发现编: {current_part}")
            i += 1
            continue
        elif line.startswith('### '):
            # 三级标题：章
            current_chapter = line[4:].strip()
            # 遇到新章时，重置节及以下标题
            current_section = ""
            current_subsection = ""
            current_subsubsection = ""
            print(f"发现章: {current_chapter}")
            i += 1
            continue

        elif line.startswith('#### '):
            # 四级标题：节
            current_section = line[5:].strip()
            # 遇到新节时，重置子节及以下标题
            current_subsection = ""
            current_subsubsection = ""
            print(f"发现节: {current_section}")
            i += 1
            continue
        elif line.startswith('##### '):
            # 五级标题：子节
            current_subsection = line[6:].strip()
            # 遇到新子节时，重置子子节
            current_subsubsection = ""
            print(f"发现子节: {current_subsection}")
            i += 1
            continue
        elif line.startswith('###### '):
            # 六级标题：子子节
            current_subsubsection = line[7:].strip()
            print(f"发现子子节: {current_subsubsection}")
            i += 1
            continue

        # 检测法条（格式：**第二百零四条** 内容...）
        # 匹配模式：**数字[零一二三四五六七八九十百千万]条?** 后面跟内容
        article_match = re.match(r'\*\*([^**]+)\*\*(.*)', line)
        if article_match:
            #提取法条编号
            article_num = article_match.group(1).strip()
            # 提取法条内容的第一行
            first_line_content = article_match.group(2).strip()

            # 收集完整的法条内容
            article_content_lines = []
            if first_line_content:
                article_content_lines.append(first_line_content)

            print(f"  发现法条: {article_num}")
            # 继续读取后续行，直到遇到下一个法条或空行
            # 继续读取后续行，收集完整的法条内容
            j = i + 1
            while j < len(lines):
                next_line = lines[j]

                # 如果遇到空行，说明法条内容结束
                if not next_line:
                    break

                # 如果遇到新的标题或新的法条，停止
                if (next_line.startswith('## ') or
                        next_line.startswith('### ') or
                        next_line.startswith('#### ') or
                        next_line.startswith('##### ') or
                        next_line.startswith('###### ') or
                        re.match(r'\*\*[^**]+\*\*', next_line)):
                    break

                # 否则是法条内容的延续
                article_content_lines.append(next_line.strip())
                j += 1

            # 合并法条内容，用空格连接
            full_content = ' '.join(article_content_lines)

            # 构建完整标题 - 按层级组合
            title_parts = []
            if current_part:
                title_parts.append(current_part)
            if current_chapter:
                title_parts.append(current_chapter)
            if current_section:
                title_parts.append(current_section)
            if current_subsection:
                title_parts.append(current_subsection)
            if current_subsubsection:
                title_parts.append(current_subsubsection)

            # 组合标题
            if title_parts:
                full_title = f"{article_num} {' - '.join(title_parts)}"
            else:
                full_title = article_num

            # 生成唯一ID（使用法条编号和内容前50字符）
            id_str = f"{article_num}_{full_content[:50]}"
            article_id = hashlib.md5(id_str.encode('utf-8')).hexdigest()[:16]

            # 保存结果
            articles.append({
                'id': article_id,
                'title': full_title,
                'content': full_content,
                'article_number': article_num,
                'part': current_part,
                'chapter': current_chapter,
                'section': current_section,
                'subsection': current_subsection,
                'subsubsection': current_subsubsection
            })

            # 跳过已处理的行
            i = j
            continue

        i += 1
    return articles

In [7]:
articles = parse_civil_code_md('mfd.md')

开始解析文件...
----------------------------------------
发现编: 中华人民共和国民法典
发现章: （二）物权编
发现节: 第一章 一般规定
  发现法条: 第二百零四条
  发现法条: 第二百零五条
  发现法条: 第二百零六条
  发现法条: 第二百零七条
  发现法条: 第二百零八条
  发现法条: 第二百零九条
  发现法条: 第二百一十条
  发现法条: 第二百一十一条
  发现法条: 第二百一十二条
  发现法条: 第二百一十三条
  发现法条: 第二百一十四条
  发现法条: 第二百一十五条
  发现法条: 第二百一十六条
  发现法条: 第二百一十七条
  发现法条: 第二百一十八条
  发现法条: 第二百一十九条
  发现法条: 第二百二十条
  发现法条: 第二百二十一条
  发现法条: 第二百二十二条
  发现法条: 第二百二十三条
  发现法条: 第二百二十四条
  发现法条: 第二百二十五条
  发现法条: 第二百二十六条
  发现法条: 第二百二十七条
  发现法条: 第二百二十八条
  发现法条: 第二百二十九条
  发现法条: 第二百三十条
  发现法条: 第二百三十一条
  发现法条: 第二百三十二条
  发现法条: 第二百三十三条
  发现法条: 第二百三十四条
  发现法条: 第二百三十五条
  发现法条: 第二百三十六条
  发现法条: 第二百三十七条
  发现法条: 第二百三十八条
  发现法条: 第二百三十九条
  发现法条: 第二百四十条
  发现法条: 第二百四十一条
  发现法条: 第二百四十二条
  发现法条: 第二百四十三条
发现节: 第二章 所有权
发现子节: 第一节 一般规定
  发现法条: 第二百四十四条
  发现法条: 第二百四十五条
  发现法条: 第二百四十四条
  发现法条: 第二百四十六条
  发现法条: 第二百四十七条
  发现法条: 第二百四十八条
  发现法条: 第二百四十九条
  发现法条: 第二百五十条
  发现法条: 第二百五十一条
  发现法条: 第二百五十二条
  发现法条: 第二百五十三条
  发现法条: 第二百五十四条
  发现法条: 第二百五十五条
  发现法条: 第二百五十六条
  发现法条: 第二百五十七条
  发现法条: 第二

In [10]:
## 读取到的articles 写入到milvus

In [8]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

In [ ]:
##默认模型总是不通，所以用一个代理中转

In [11]:
!pip install "numpy<2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 10.8 MB/s  0:00:011.1 MB/s eta 0:00:01:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.4.2
    Uninstalling numpy-2.4.2:
      Successfully uninstalled numpy-2.4.2


In [12]:
from pymilvus import model as milvus_model
embedding_model = milvus_model.dense.OpenAIEmbeddingFunction(
    model_name='text-embedding-3-large', # Specify the model name
    api_key= 'sk-XNnNtPyKjhNwYLOjA13f5b2a5b044fA7987249044cD18eF6', # Provide your OpenAI API key
    base_url='https://api.apiyi.com/v1',
    dimensions=512
)

In [13]:
test_embedding = embedding_model.encode_queries(["民法典有没有规定贷款的相关要求"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

512
[ 0.01172358 -0.01499171 -0.0182708   0.09194621 -0.02781197 -0.0368048
  0.00699138  0.01252416 -0.01345634 -0.02550893]


In [ ]:
##创建桶

In [14]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_demo.db")

collection_name = "mfd_rag_collection"

/Users/allanqin/anaconda3/envs/py11/lib/python3.11/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [15]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

In [16]:
if milvus_client.has_collection(collection_name):
    print('the collection is already exist')

In [17]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

In [18]:
if milvus_client.has_collection(collection_name):
    print('the collection is already exist')

the collection is already exist


In [ ]:
## 插入数据

In [19]:
print(articles[0])

{'id': 'b6763aae3eabe317', 'title': '第二百零四条 中华人民共和国民法典 - （二）物权编 - 第一章 一般规定', 'content': '为了明确物的归属，充分发挥物的效用，保护权利人的合法权益，维护社会经济秩序，制定本编。', 'article_number': '第二百零四条', 'part': '中华人民共和国民法典', 'chapter': '（二）物权编', 'section': '第一章 一般规定', 'subsection': '', 'subsubsection': ''}


In [20]:
def str_to_int_id(string_id):
    """将字符串ID转换为整数"""
    # 使用hashlib生成一个整数
    return int(hashlib.md5(string_id.encode()).hexdigest()[:8], 16)

In [21]:
from tqdm import tqdm
import json

# 准备要编码的文本（将title和content组合）
texts_to_encode = []
for article in articles:
    # 组合标题和内容作为向量化的依据
    combined_text = f"{article['title']} {article['content']}"
    texts_to_encode.append(combined_text)

print(f"准备编码 {len(texts_to_encode)} 条文本...")

# 批量编码所有文本
doc_embeddings = embedding_model.encode_documents(texts_to_encode)

# 准备插入Milvus的数据
data = []
for i, article in enumerate(tqdm(articles, desc="Preparing data for Milvus")):
    int_id = str_to_int_id(article['id'])
    current_text = f"{article['title']} {article['content']}"
    data.append({
        "id": int_id,  # 使用原始ID，而不是索引
        "article_number": article['article_number'],
        "title": article['title'],
        "content": article['content'],
        "part": article['part'],
        "chapter": article['chapter'],
        "section": article['section'],
        "subsection": article['subsection'],
        "subsubsection": article['subsubsection'],
        "vector": doc_embeddings[i],
        "text": current_text  # 可选，如果需要存储完整文本
    })

print(f"开始插入 {len(data)} 条数据到Milvus...")
milvus_client.insert(collection_name=collection_name, data=data)
print("插入完成！")

准备编码 387 条文本...


Preparing data for Milvus: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 387/387 [00:00<00:00, 206382.15it/s]

开始插入 387 条数据到Milvus...
插入完成！


In [22]:
question_fist = "别人借钱不还，怎么办?"

In [ ]:
## 第一个问题

In [23]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question_fist]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

In [24]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4,ensure_ascii=False))

[
    [
        "第五百八十五条 中华人民共和国民法典 - （三）合同编 - 第五章 违约责任 当事人约定金钱债务的，一方不履行债务或者履行债务不符合约定，致使不能实现合同目的的，对方可以请求支付违约金。",
        0.48194634914398193
    ],
    [
        "第五百八十七条 中华人民共和国民法典 - （三）合同编 - 第五章 违约责任 债务人履行债务时，定金应当抵作价款或者收回。给付定金的一方不履行债务或者履行债务不符合约定，致使不能实现合同目的的，无权请求返还定金；收受定金的一方不履行债务或者履行债务不符合约定，致使不能实现合同目的的，应当双倍返还定金。",
        0.47498852014541626
    ],
    [
        "第五百三十六条 中华人民共和国民法典 - （三）合同编 - 第二章 合同的履行 债务人怠于行使其对第三人的债权或者与该债权有关的从权利，影响债权人的到期债权实现的，债权人可以向人民法院请求以自己的名义代位行使债务人的债权。",
        0.47265124320983887
    ]
]


In [ ]:
大模型处理，转换为字符串连接起来，将上述回答的三个结果的内容

In [25]:
context1 = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [26]:
context1

'第五百八十五条 中华人民共和国民法典 - （三）合同编 - 第五章 违约责任 当事人约定金钱债务的，一方不履行债务或者履行债务不符合约定，致使不能实现合同目的的，对方可以请求支付违约金。\n第五百八十七条 中华人民共和国民法典 - （三）合同编 - 第五章 违约责任 债务人履行债务时，定金应当抵作价款或者收回。给付定金的一方不履行债务或者履行债务不符合约定，致使不能实现合同目的的，无权请求返还定金；收受定金的一方不履行债务或者履行债务不符合约定，致使不能实现合同目的的，应当双倍返还定金。\n第五百三十六条 中华人民共和国民法典 - （三）合同编 - 第二章 合同的履行 债务人怠于行使其对第三人的债权或者与该债权有关的从权利，影响债权人的到期债权实现的，债权人可以向人民法院请求以自己的名义代位行使债务人的债权。'

In [31]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。
<context>
{context1}
</context>
<question>
{question_fist}
</question>
"""

In [32]:
## 大模型回答结果
response1 = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response1.choices[0].message.content)

根据《中华人民共和国民法典》的相关规定，如果别人借钱不还，您可以采取以下法律途径：

1. **请求支付违约金**：如果借款合同中约定了违约金，对方不履行债务导致合同目的无法实现，您可以依据《民法典》第五百八十五条请求对方支付违约金。

2. **定金条款适用**：如果借款涉及定金，根据《民法典》第五百八十七条，若对方不履行债务，您作为收受定金方有权不返还定金；若您支付了定金而对方违约，对方应双倍返还定金。

3. **行使代位权**：如果债务人怠于行使其对第三人的债权，影响您的债权实现，您可以依据《民法典》第五百三十六条向人民法院请求以自己的名义代位行使债务人的债权。

建议先与对方协商，协商不成时，可依据上述条款向人民法院提起诉讼，以维护您的合法权益。


In [33]:
question_two = "偷钱如何处理?"

In [34]:
search_res2 = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question_two]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

In [35]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res2[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4,ensure_ascii=False))

[
    [
        "第二百三十六条 中华人民共和国民法典 - （二）物权编 - 第一章 一般规定 权利人悬赏寻找遗失物的，领取遗失物时应当按照承诺向拾得人支付报酬。 拾得人侵占遗失物的，无权请求保管费等费用，也无权请求权利人支付报酬。",
        0.46390724182128906
    ],
    [
        "第三百五十条 中华人民共和国民法典 - （二）物权编 - 第二章 所有权 - 第四节 所有权取得的特别规定 权利人悬赏寻找遗失物的，领取遗失物时应当按照承诺向拾得人支付报酬。 拾得人侵占遗失物的，无权请求保管费等费用，也无权请求权利人支付报酬。",
        0.45410650968551636
    ],
    [
        "第二百三十四条 中华人民共和国民法典 - （二）物权编 - 第一章 一般规定 拾得遗失物，应当返还权利人。拾得人应当及时通知权利人领取，或者送交公安等有关部门。",
        0.40812450647354126
    ]
]


In [1]:
## 大模型读取检索结果给出答案

In [2]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

NameError: name 'retrieved_lines_with_distances' is not defined